In [ ]:
#세이지 메이커 최신 업데이트
!pip3 install -U sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
import os
import boto3
import re
import sagemaker

#서비스를 이용하기 위한 권한 ,지역 , s3 주소 설정.
role = sagemaker.get_execution_role() #권한 노트북 or 세이지메이커 세션에 연결된 권한으로 권한 연결
region = sagemaker.Session().boto_region_name #지역 지역 설정으로 boto가 연결된(사용할) 지역으로 지역 설정
bucket = sagemaker.Session().default_bucket() #버켓  기본 버켓 설정 버킷이름에 지역+사용자이름의 버킷을 자동생성

prefix = "4N/TEST4"

#s3이름은
#sagemaker-ap-northeast-2-197123..../4N/TEST4

In [ ]:
#간단하게 업로드는 하는 함수만 사용함

s3 = boto3.client("s3")
#s3에 파일 업로드
s3.upload_file("data/abalone.train_0", bucket, prefix + "/"+ "train/train_0.libsvm")
s3.upload_file("data/abalone.validation", bucket, prefix + "/"+ "validation/validation.libsvm")
s3.upload_file("data/abalone.test", bucket, prefix + "/"+ "test/test.libsvm")

아래 script1.py는 xgb 함수에 .ArgumentParser를 이용해 파라미터들을 받아 트레이닝합니다.
def _xgb_train():의  xgb.train는 기존 xgb 방법과 같고 def _xgb_train()에 그 파라미터와 저장할 곳(model_dir)을 추가한 함수입니다.

In [ ]:
%%writefile ./script1.py

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl

from sagemaker_containers import entry_point #교육 또는 추론을 위한 컨테이너 이미지를 생성하는 데 사용됩니다.
from sagemaker_xgboost_container.data_utils import get_dmatrix #dmatrix 형태
from sagemaker_xgboost_container import distributed

import xgboost as xgb


def _xgb_train(params, dtrain, evals, num_boost_round, model_dir):

    model = xgb.train(params=params,
                        dtrain=dtrain,
                        evals=evals,
                        num_boost_round = num_boost_round)

    model_location = model_dir + '/xgboost-model'
    pkl.dump(model, open(model_location, 'wb'))



if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # XGB의 파라미터
    parser.add_argument('--num_round', type=int)
    parser.add_argument('--max_depth', type=int)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--gamma', type=int)
    parser.add_argument('--min_child_weight', type=int)
    parser.add_argument('--subsample', type=float)
    parser.add_argument('--seed', type=int)


    #경로 파라미터
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR")) #경로
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))


    args, _ = parser.parse_known_args()

    #XGB 사용법과 동일 데이터를 경로를 통해 받고
    dtrain = get_dmatrix(args.train, 'libsvm')
    dval = get_dmatrix(args.validation, 'libsvm')
    watchlist = [(dtrain, 'train'), (dval, 'validation')]

    #파라미터 넣어주고
    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'gamma': args.gamma,
        'min_child_weight': args.min_child_weight,
        'subsample': args.subsample,
    }

    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir)

    #training
    _xgb_train(**xgb_train_args)



Overwriting ./script1.py


In [ ]:
#하이퍼파라미터 그 외 환경설정
hyperparams = {
    "num_round":"20",
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "seed": "42"
}

instance_type = "ml.m5.2xlarge"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "result")
#4N/TEST/result/...
content_type = "libsvm"

In [ ]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

session = Session()

# 총 환경설정
xgb_script_mode_estimator = XGBoost(
    entry_point="script1.py",
    framework_version="1.7-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",#instance_type
    output_path=output_path,#4N/TEST/result/...
)
# 데이터 주소를 담음
train_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "train"), content_type=content_type
)
validation_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "validation"), content_type=content_type
)

In [ ]:
xgb_script_mode_estimator.fit({"train": train_input, "validation": validation_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-17-04-03-59-190


Using provided s3_resource
2023-08-17 04:03:59 Starting - Starting the training job...
2023-08-17 04:04:15 Starting - Preparing the instances for training......
2023-08-17 04:05:24 Downloading - Downloading input data
2023-08-17 04:05:24 Training - Downloading the training image...
2023-08-17 04:05:44 Training - Training image download completed. Training in progress...[2023-08-17 04:06:08.341 ip-10-0-166-93.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-08-17 04:06:08.363 ip-10-0-166-93.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-08-17:04:06:08:INFO] Imported framework sagemaker_xgboost_container.training
[2023-08-17:04:06:08:INFO] No GPUs detected (normal if no gpus installed)
[2023-08-17:04:06:08:INFO] Invoking user training script.
[2023-08-17:04:06:08:INFO] Module script1 does not provide a setup.py. 
Generating setup.py
[2023-08-17:04:06:08:INFO] Generating setup.cfg
[2023-0

[0]#011train-rmse:8.16225#011validation-rmse:7.98988
[1]#011train-rmse:6.68992#011validation-rmse:6.54152
[2]#011train-rmse:5.52560#011validation-rmse:5.40125
[3]#011train-rmse:4.62792#011validation-rmse:4.53032
[4]#011train-rmse:3.92357#011validation-rmse:3.84269
[5]#011train-rmse:3.37277#011validation-rmse:3.33351
[6]#011train-rmse:2.96267#011validation-rmse:2.95324
[7]#011train-rmse:2.64494#011validation-rmse:2.69374
[8]#011train-rmse:2.42382#011validation-rmse:2.50521
[9]#011train-rmse:2.25485#011validation-rmse:2.37596
[10]#011train-rmse:2.14524#011validation-rmse:2.29647
[11]#011train-rmse:2.05020#011validation-rmse:2.23640
[12]#011train-rmse:1.99161#011validation-rmse:2.21536
[13]#011train-rmse:1.94721#011validation-rmse:2.19622
[14]#011train-rmse:1.90350#011validation-rmse:2.18586
[15]#011train-rmse:1.86750#011validation-rmse:2.17532
[16]#011train-rmse:1.84277#011validation-rmse:2.17141
[17]#011train-rmse:1.81594#011validation-rmse:2.15823
[18]#011train-rmse:1.80284#011validation-rmse:2.15822
[19]#011train-rmse:1.79006#011validation-rmse:2.16690

로그에 보시면 num_round가 20이므로 rmse가 20번 출력되었습니다.
이제 아래셀에서는 script2.py 문에서 10번(로그[0]~[9]번 rmse) 트레이닝 진행 후 모델 저장 그리고 script3.py에서 모델을 받아서 10번([10]~[19])을 트레이닝을 진행할 것입니다.(script2.py와 script3.py은 동일한 script문입니다.)
script3.py 트레이닝 함수안에 if문을로 만들어 모델 주소가 없으면 그냥 위의 script1.py 처럼 트레이닝을 진행하고 모델 파라미터를 입력해주면 if문을 통해 모델을 로드하고 트레이닝을 이어서 하도록 구성하였습니다.

파라미터에 모델 url만 넣어주면 작동

In [ ]:
%%writefile ./script2.py

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl

from sagemaker_containers import entry_point #교육 또는 추론을 위한 컨테이너 이미지를 생성하는 데 사용됩니다.
from sagemaker_xgboost_container.data_utils import get_dmatrix #dmatrix 형태
from sagemaker_xgboost_container import distributed

import xgboost as xgb


import boto3
import tarfile
import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl
from urllib.parse import urlparse

from sagemaker_containers import entry_point #교육 또는 추론을 위한 컨테이너 이미지를 생성하는 데 사용됩니다.
from sagemaker_xgboost_container.data_utils import get_dmatrix #dmatrix 형태
from sagemaker_xgboost_container import distributed

import xgboost as xgb

def download_from_s3(s3_url, local_path): #모델을 인스턴스 가상환경에 업로드 하기 위한 함수
    """Download a file from S3 to the local file system"""
    parsed_url = urlparse(s3_url)
    bucket_name = parsed_url.netloc
    s3_path = parsed_url.path.lstrip('/')
    boto3.resource('s3').Bucket(bucket_name).download_file(s3_path, local_path)


def _xgb_train(params, dtrain, evals,num_boost_round, model_dir, model_location):

    if model_location: #모델의 url이 있으면

        download_from_s3(args.model_location, '/tmp/model.tar.gz') #모델을 다운로드


        with tarfile.open('/tmp/model.tar.gz', 'r:gz') as file:
            file.extractall('/tmp/')

        loaded_model = pkl.load(open('/tmp/xgboost-model', 'rb')) # 모델을 담고

        model = xgb.train(params=params,
                            dtrain=dtrain,
                            evals=evals,
                            num_boost_round = num_boost_round,
                            xgb_model=loaded_model) #retrain 파라미터 사용

        model_location = model_dir + '/xgboost-model'
        pkl.dump(model, open(model_location, 'wb'))

    else: # script1.py과 동일

        model = xgb.train(params=params,
                            dtrain=dtrain,
                            evals=evals,
                            num_boost_round = num_boost_round)

        model_location = model_dir + '/xgboost-model'
        pkl.dump(model, open(model_location, 'wb'))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # XGB의 파라미터
    parser.add_argument('--num_round', type=int)
    parser.add_argument('--max_depth', type=int)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--gamma', type=int)
    parser.add_argument('--min_child_weight', type=int)
    parser.add_argument('--subsample', type=float)
    parser.add_argument('--seed', type=int)

    #train,validation 파일이 있는 위치
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR")) #경로
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--model_location', type=str, default="") #모델 url을 받을 파라미터


    args, _ = parser.parse_known_args()

    dtrain = get_dmatrix(args.train, 'libsvm')
    dval = get_dmatrix(args.validation, 'libsvm')
    watchlist = [(dtrain, 'train'), (dval, 'validation')]

    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'gamma': args.gamma,
        'min_child_weight': args.min_child_weight,
        'subsample': args.subsample,
    }

    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir,
        model_location = args.model_location,
        )

    #training
    _xgb_train(**xgb_train_args)

Overwriting ./script2.py


In [ ]:
#하이퍼파라미터 그 외 환경설정
hyperparams = {
    "num_round":"10",
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "seed": "42"

}

instance_type = "ml.m5.2xlarge"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "result")
#4N/TEST/result/...
content_type = "libsvm"

In [ ]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

session = Session()

# 총 환경설정
xgb_script_mode_estimator = XGBoost(
    entry_point="script2.py",
    framework_version="1.7-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",#instance_type
    output_path=output_path,#4N/TEST/result/...
)
# 데이터 주소를 담음
train_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "train"), content_type=content_type
)
validation_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "validation"), content_type=content_type
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.2xlarge.


In [ ]:
xgb_script_mode_estimator.fit({"train": train_input, "validation": validation_input})

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-17-04-06-41-643


2023-08-17 04:06:41 Starting - Starting the training job...
2023-08-17 04:06:56 Starting - Preparing the instances for training......
2023-08-17 04:07:48 Downloading - Downloading input data...
2023-08-17 04:08:28 Training - Training image download completed. Training in progress...[2023-08-17 04:08:46.974 ip-10-0-175-247.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-08-17 04:08:46.997 ip-10-0-175-247.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-08-17:04:08:47:INFO] Imported framework sagemaker_xgboost_container.training
[2023-08-17:04:08:47:INFO] No GPUs detected (normal if no gpus installed)
[2023-08-17:04:08:47:INFO] Invoking user training script.
[2023-08-17:04:08:47:INFO] Module script2 does not provide a setup.py. 
Generating setup.py
[2023-08-17:04:08:47:INFO] Generating setup.cfg
[2023-08-17:04:08:47:INFO] Generating MANIFEST.in
[2023-08-17:04:08:47:INFO] Installing module

[0]#011train-rmse:8.16225#011validation-rmse:7.98988
[1]#011train-rmse:6.68992#011validation-rmse:6.54152
[2]#011train-rmse:5.52560#011validation-rmse:5.40125
[3]#011train-rmse:4.62792#011validation-rmse:4.53032
[4]#011train-rmse:3.92357#011validation-rmse:3.84269
[5]#011train-rmse:3.37277#011validation-rmse:3.33351
[6]#011train-rmse:2.96267#011validation-rmse:2.95324
[7]#011train-rmse:2.64494#011validation-rmse:2.69374
[8]#011train-rmse:2.42382#011validation-rmse:2.50521
[9]#011train-rmse:2.25485#011validation-rmse:2.37596

로그가 10번 출력되었습니다.

이제 모델 url을 넣어 이어서 트레이닝해보겠습니다. url은 하이퍼파라미터로 전달하였습니다.
아래 script3.py는 위의 script2.py와 같습니다.

In [ ]:
%%writefile ./script3.py

import boto3
import tarfile
import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl
from urllib.parse import urlparse

from sagemaker_containers import entry_point #교육 또는 추론을 위한 컨테이너 이미지를 생성하는 데 사용됩니다.
from sagemaker_xgboost_container.data_utils import get_dmatrix #dmatrix 형태
from sagemaker_xgboost_container import distributed

import xgboost as xgb

def download_from_s3(s3_url, local_path):
    """Download a file from S3 to the local file system"""
    parsed_url = urlparse(s3_url)
    bucket_name = parsed_url.netloc
    s3_path = parsed_url.path.lstrip('/')
    boto3.resource('s3').Bucket(bucket_name).download_file(s3_path, local_path)


def _xgb_train(params, dtrain, evals,num_boost_round, model_dir, model_location):

    if model_location:

        download_from_s3(args.model_location, '/tmp/model.tar.gz')


        with tarfile.open('/tmp/model.tar.gz', 'r:gz') as file:
            file.extractall('/tmp/')

        loaded_model = pkl.load(open('/tmp/xgboost-model', 'rb'))

        model = xgb.train(params=params,
                            dtrain=dtrain,
                            evals=evals,
                            num_boost_round = num_boost_round,
                            xgb_model=loaded_model)

        model_location = model_dir + '/xgboost-model'
        pkl.dump(model, open(model_location, 'wb'))

    else:

        model = xgb.train(params=params,
                            dtrain=dtrain,
                            evals=evals,
                            num_boost_round = num_boost_round)

        model_location = model_dir + '/xgboost-model'
        pkl.dump(model, open(model_location, 'wb'))



if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # XGB의 파라미터
    parser.add_argument('--num_round', type=int)
    parser.add_argument('--max_depth', type=int)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--gamma', type=int)
    parser.add_argument('--min_child_weight', type=int)
    parser.add_argument('--subsample', type=float)
    parser.add_argument('--seed', type=int)


    #train,validation 파일이 있는 위치
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR")) #경로
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--model_location', type=str, default="")


    args, _ = parser.parse_known_args()

    dtrain = get_dmatrix(args.train, 'libsvm')
    dval = get_dmatrix(args.validation, 'libsvm')
    watchlist = [(dtrain, 'train'), (dval, 'validation')]

    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'gamma': args.gamma,
        'min_child_weight': args.min_child_weight,
        'subsample': args.subsample,
    }

    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir,
        model_location = args.model_location,
        )

    #training
    _xgb_train(**xgb_train_args)

Overwriting ./script3.py


In [ ]:
#하이퍼파라미터 그 외 환경설정

hyperparams = {
    "num_round":"10",
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "seed": "42",
    "model_location": 's3://sagemaker-ap-northeast-2-197354151997/4N/TEST4/result/output/sagemaker-xgboost-2023-08-17-04-06-41-643/output/model.tar.gz'
    }
#모델 url을 model_location을 통해 전달해 줍니다.

instance_type = "ml.m5.2xlarge"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "result")
#4N/TEST/result/...
content_type = "libsvm"

In [ ]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

session = Session()

# 총 환경설정
xgb_script_mode_estimator = XGBoost(
    entry_point="script3.py",
    framework_version="1.7-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",#instance_type
    output_path=output_path,#4N/TEST/result/...
)
# 데이터 주소를 담음
train_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "train"), content_type=content_type
)
validation_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "validation"), content_type=content_type
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.2xlarge.


In [ ]:
xgb_script_mode_estimator.fit({"train": train_input, "validation": validation_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-17-04-09-23-992


Using provided s3_resource
2023-08-17 04:09:24 Starting - Starting the training job...
2023-08-17 04:09:40 Starting - Preparing the instances for training......
2023-08-17 04:10:51 Downloading - Downloading input data
2023-08-17 04:10:51 Training - Downloading the training image...
2023-08-17 04:11:11 Training - Training image download completed. Training in progress..[2023-08-17 04:11:28.569 ip-10-0-172-221.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-08-17 04:11:28.592 ip-10-0-172-221.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-08-17:04:11:28:INFO] Imported framework sagemaker_xgboost_container.training
[2023-08-17:04:11:28:INFO] No GPUs detected (normal if no gpus installed)
[2023-08-17:04:11:28:INFO] Invoking user training script.
[2023-08-17:04:11:29:INFO] Module script3 does not provide a setup.py. 
Generating setup.py
[2023-08-17:04:11:29:INFO] Generating setup.cfg
[2023-

[0]#011train-rmse:2.14056#011validation-rmse:2.30779
[1]#011train-rmse:2.05782#011validation-rmse:2.24896
[2]#011train-rmse:1.98323#011validation-rmse:2.20880
[3]#011train-rmse:1.93551#011validation-rmse:2.18483
[4]#011train-rmse:1.89966#011validation-rmse:2.17883
[5]#011train-rmse:1.87614#011validation-rmse:2.18025
[6]#011train-rmse:1.83646#011validation-rmse:2.18298
[7]#011train-rmse:1.82295#011validation-rmse:2.18279
[8]#011train-rmse:1.81029#011validation-rmse:2.18358
[9]#011train-rmse:1.79925#011validation-rmse:2.18429

로그가 10번 출력되었습니다.

script1.py에서 20번 출력된 결과물과 script2.py(10번) + script3.py(10번)에서 이어서 트레이닝된 부분을 비교해서 확인할 수 있습니다.
rmse 수치가 완벽히 동일하지 않지만 모델에 무엇을 저장하느냐에 따라 또는 함수에 관련된 초기화와 잠재적 요인을 통일한다면 같은 값을 가질 것입니다.
모델을 이어서 트레이닝하는 부분을 여러분들이 만드신 ESN pytorch에서 구성한다고 가정하면 특정 epoch가 반복될 때마다 model을 저장하고 그리고 위의 script 코드처럼 if문을 구성하고 url을 전달해 준다면 세이지메이커에서 재트레닝도 할 수 있는 script가 만들어집니다.